# Simulus Tutorial

This tutorial describes the basic functions of simulus. You should be able to find all examples shown in this tutorial under the `examples` directory in the simulus source-code distribution.

## 1.  What's Simulus?

Simulus is a process-oriented discrete-event simulator written in Python. That's quite a mouthful. Let's first break it down what that really means. 

A discrete-event simulator models the world as a sequence of events that happen in discrete time. The simulator basically processes these events in order. Such a simulator would keep a simulation clock, which advances its time (by leaps and bounds) in accordance with the time of the events being processed along the way.

For example, a professor starts the day with a cup of coffee at 4 AM, reads for 2 hours, has breakfast, takes shower, and then drives to school at 7:30 AM. The discrete-event simulator for the professor's day will go through a sequence of events: a "wake up" event at 4:00, a "start coffee" event at 4:05, a "start read" event at 4:10, a "finish coffee" event at 4:20, a "finish read" event at 6:10, a "breakfast" event at 6:30, a "shower" event at 6:50, a "start driving" event at 7:30, so on and so forth. The clock of the simulator in this case will go through a sequence of increasing values: 4:00, 4:05, 4:10, ... You get the gist. 

Discrete-event simulation is a very powerful modeling method. You can think of the entire world, including the activities of the professors, the students, and all other people, as a sequence of events that happen over time. The simulator's job is to "play out" the events in order. By doing so, we hopefully get better understanding of the world.

Simulus is a "process-oriented" discrete-event simulator. What it means is that in addition to describing the world as a sequence of events, one can also model it using processes. As an example, suppose the world consists of many professors, students, and people. Previously we described the schedule of one professor. A different person would have a different schedule and follow a different sequence of events. Using process-oriented simulation, every person in this system can be modeled as a separate process or processes. 

Note that people interact in this world. For example, if everyone starts driving to school or to work in the morning at the same hour, most may get delayed since congestion would happen on the road. Also, if a student needs to meet with another student at a coffee shop. They would not be able to carry on with their next task (say, to work on their joint project) until both of them are present in the coffee shop. Simulus provides the needed support for creating and managing the processes, and having them coordinate with one another, so as to make it easier for you to model the complexed interactions and procedures in this world.

## 2.  How Does It Work?

Simulus works in two ways. One way is through events. You can schedule events and process them. We call it *direct event scheduling*. The other way is through processes. You can create processes and have them run and interact. We call it *process scheduling*. Of course, there's a third way, by combining them and having both event schedule and process scheduling in the models.

### 2.1  Direct Event Scheduling

In simulus, an event is simply a function to be invoked at a designated time. This can be illustrated using the following hello-world example.

#### 2.1.1  The Hello-World Example

In [1]:
# %load "../examples/basics/helloworld.py"
import simulus

def print_message():
    print("Hello world at time", sim.now)
    
sim = simulus.simulator()
sim.sched(print_message, until=10)
sim.run()


Hello world at time 10


In order to use the simulator, we need to first import the `simulus` package and then create a simulator using the `simulator()` function. A simulator maintains an event list where all events will be sorted in their timestamp order. A simulator also keeps the current simulation time, which gets advanced while executing the events on the event list. One can find out the current time of the simulator by inspecting the `now` variable of the simulator.

In the example, the function `print_message()` is scheduled to run at time 10. We schedule the function using the `sched()` method of the simulator, and passing the name of the function with the `until` argument to specify the time at which the function should be invoked. In discrete-event simulation terminology, the `print_message()` function is also called an *event handler*. In simulus, an event handler can be any Python function or method.

To run the simulation, we simply use the `run()` method of the simulator. Without an argument, the `run()` method will process *all* events on the event list; the function returns only when there are no more events on the event list.

#### 2.1.2  Passing Arguments to Event Handlers

The next example shows how to pass arguments between `sched()` and the event handler. 

In [2]:
# %load "../examples/basics/passargs.py"
import simulus

def print_params_1(a, b, c="named"):
    print("print_params_1(a=%r, b=%r, c=%r) invoked at time %g" %
          (a, b, c, sim.now))

def print_params_2(mysim, a, b, *args, c="named", **kwargs):
    print("print_params_2(a=%r, b=%r, args=%r, c=%r, kwargs=%r) invoked at time %g" %
          (a, b, args, c, kwargs, mysim.now))

class myclass:
    def __init__(self, a):
        self.a = a

    def print_params_3(self, b, c):
        print("print_params_3(a=%r, b=%r, c=%r) invoked at %g" %
              (self.a, b, c, sim.now))
    
sim = simulus.simulator()
sim.sched(print_params_1, "hello", 100, until=10)
sim.sched(print_params_2, sim, "hello", 100, "yes", "no", arg1=True, arg2=2, c=1, until=20)
cls = myclass(10)
sim.sched(myclass.print_params_3, cls, 11, 12, until=30)
sim.sched(cls.print_params_3, 11, 12, until=40)
sim.run()


print_params_1(a='hello', b=100, c='named') invoked at time 10
print_params_2(a='hello', b=100, args=('yes', 'no'), c=1, kwargs={'arg1': True, 'arg2': 2}) invoked at time 20
print_params_3(a=10, b=11, c=12) invoked at 30
print_params_3(a=10, b=11, c=12) invoked at 40


In the previous hello-world example, we did not pass any arguments to the `print_params()` function. The `sim` variable is module scoped. That is, as long as the function stays in the same module where `sim` is defined, it'll be fine. But this would not work in a more complicated scenario, e.g., when we develop a model that spreads into multiple python modules, or when the functions need paramesters.

In this example, we schedule three functions: `print_params_1()` at time 10, `print_params_1()` at time 20, and `print_params_3()` at both time 30 and time 40. We can pass aguments from `sched()` to an event handler by placing the arguments right after the function name (the first argument of `sched()`). Here we take advantage of Python's ability to take both positional arguments and keyworded arguments. In Python, the single-asterisk form of `*args` can be used to take a non-keyworded variable-length list of arguments passed to a function. The double asterisk form of `**kwargs` can be used to take a keyworded, variable-length dictionary of arguments passed to a function. 

In `print_params_1()`, the variables `a` and `b` are positional arguments, and `c` is a keyworded argument. In `print_params_2()`, the variables `mysim`, `a`, and `b` are three positional arguments, which take `sim`, the string "hello", and the value 100 from the call to `sched()`. The additional positional arguments, "yes" and "no", are placed into the list `args`. The variable `c` is a keyworded argument. The rest of the keyworded arguments are placed into the dictionary `kwargs`. 

`print_params_3()` is a method inside `myclass`. We show two ways to identify it as an event handler. One way is pass the class method as the name of the function, followed by the class instance as the first positional argument. The other way is to directly use the method of the specific class instance. Python is smart enough to unpack it to be the class method and the class instance when `sched()` is called.

The `sched()` function takes its own arguments. They are all keyworded arguments, such as `until`. The `sched()` function will first filter out the recognized keyworded arguments for itself, before scheduling the user intended function to be invoked at the designated time. All positional arguments and all unfiltered keyworded arguments are passed onto the event handler.

In Python, the parameters of a function need to occur in a particular order: first, the formal positional arguments, then `*args`, followed by the keyworded arguments, and finally `**kwargs`. That's exactly what we did for the `print_params_2` function.

#### 2.1.3  The Life of a Professor

In simulus, one can of course schedule more functions to run within an event handler. The whole point of simulation is to help capture and examine the complicated logic or processes of the world (or system) as the events unfold. 

In the next example, we show the "complicated" life of a professor.

In [3]:
# %load "../examples/basics/professor.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up():
    print("professor wakes up at "+strnow())
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee():
    print("professor starts drinking coffee at "+strnow())
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee():
    print("professor finishes drinking coffee at "+strnow())
    
def start_read():
    print("professor starts reading at "+strnow())
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read():
    print("professor finishes reading at "+strnow())

def breakfast():
    print("professor breakfasts at "+strnow())

def shower():
    print("professor shows at "+strnow())

def leave():
    print("professor leaves home and drives to school at "+strnow())
    sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive():
    print("professor arrives at school at "+strnow())

def meeting1():
    print("professor has first meeting at "+strnow())

def meeting2():
    print("professor has second meeting at "+strnow())

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600) # 4:00
sim.sched(meeting1, until=9*3600) # 9:00
sim.sched(meeting2, until=10*3600) # 10:00
sim.run()


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor shows at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor has first meeting at 09:00:00
professor has second meeting at 10:00:00


This example is mostly self-explanatory. There's but one subtle point. The simulation time in simulus is represented as an integer or a floating point number. In this example, we represent time in seconds from midnight. To make things a bit more comprehensible for the printout, in this example we use the standard `time` module in Python to convert the seconds into human readble formats: hours, minutes, and seconds.

#### 2.1.4  Canceling and Rescheduling Events

So far, the professor's schedule is rather boring. In real life, we make appointments and we cancel them sometimes. We also reschedule appointments. Let's make things a bit more interesting for the professor's life. Let's assume that today there's a big traffic jam. So instead of taking only 45 minutes, the professor takes 2 hours 45 minutes to get to school. Obviously, the professor will miss the first meeting and get late for the second one. So she cancels the first meeting and reschedules the second meeting. 

To do that we need to use the return value from the `sched()` method. It is actually a reference to an event that's got scheduled onto the simulator's event list. Event in simulus is an opaque object. That is, we can make a reference to the event (in order to cancel or reschedule the event), but we should not directly access the member variables and methods within the event instance. In this example, we call the simulator's `cancel()` and `resched()` methods and pass the event as the argument. The professor cancels and reschedules the events when she is about to leave home (assuming she can predict that the traffic jam is about to happen by then).

Another difference of this example from the previous one is that we make professor's waking up a daily repeatable event (with a 24 hour interval). In this case, the professor always gets up at 4 AM every day and carries out the same set of activities (my goodness!). If we `run()` the simulation without an argument, it would never return as there will always be events on the event list. Because of that, in this example, we call `run()` with a designated simulation end-time using the keyworded argument `until`. Specifically for this example, we run the simulation for a duration of three days.

In [4]:
# %load "../examples/basics/professor-flex.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up():
    print("professor wakes up at "+strnow())
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee():
    print("professor starts drinking coffee at "+strnow())
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee():
    print("professor finishes drinking coffee at "+strnow())
    
def start_read():
    print("professor starts reading at "+strnow())
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read():
    print("professor finishes reading at "+strnow())

def breakfast():
    print("professor breakfasts at "+strnow())

def shower():
    print("professor showers at "+strnow())

def leave():
    print("professor leaves home and drives to school at "+strnow())
    if sim.now < 24*3600:
        # traffic jam at the first day
        sim.sched(arrive, offset=2*3600+45*60) # 2 hours and 45 minutes from now
        # the two meetings are only at the first day
        sim.cancel(e1)
        sim.resched(e2, until=11*3600) # 11:00
    else:
        # no traffic jam in the following days
        sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive():
    print("professor arrives at school at "+strnow())

def meeting1():
    print("professor has first meeting at "+strnow())

def meeting2():
    print("professor has second meeting at "+strnow())

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600, repeat_intv=24*3600) # 4:00 
e1 = sim.sched(meeting1, until=9*3600) # 9:00
e2 = sim.sched(meeting2, until=10*3600) # 10:00
sim.run(until=72*3600)


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

#### 2.1.5  More on Simulators

By default, a simulator starts at time zero. Instead, we can use the `init_time` argument to change the start simulation time when we create the simulator. 

A simulator can also have an optional name using the `name` argument. If specified, the simulator's name should be unique. In this case, one can use the `get_simulator()` function to retrieve the simulator using the name. If the name is not unique, a simulator created at a later time will replace the one created earlier with the same name. In that case, the earlier simulator will no longer be retrievable using the name. A simulator's name is optional. A simulator can be created without a name. In this case, one cannot use the `get_simulator()` function to retrieve it. Giving a simulator a name can be useful when we deal with distributed simulation. We will discuss it more when we get to simulus' parallel and distributed simulation support. 

All the previous examples create had one simulator at a time. In fact, one can simultaneously run multiple simulators, each maintaining its own event list and its own current simulation time. If multiple simulators are created, they will all run independently. The event handlers at different simulators are invoked in separate timelines. That is, the current time at one simulator has no relation to the current time of another simulator. This is the default behavior. Simulators can also be time synchronized (using the `sync()` function). We will return to this topic later when we discuss the parallel and distributed simulation support of simulus.

Each simulator processes its own events when we call the `run()` function. We can specify either `offset` or `until` when we call `run()`, in which case the simulator will process all events with timestamps no larger than the designated time. The `offset` argument specifies a relative time from the simulator's current simulation time. The `until` argument specifies an absolute time. When the `run()` function returns, the current time of the simulator will be advanced to the designated time and all events on the event list with timestamps smaller than or equal to the designated time have already been processed. If both `offset` and `until` are ignored, the simulator will run as long as there are events on the event list. The user should be aware that, in this case, a simulator may run forever as in some simulation models there are always new events generated when the event handlers are invoked (like in the previous example).

We can also step through simulation by processing one event at a time. This is achieved by the simulator's `step()` function, which only processes the next event on the event list and advances the simulation time to the time of the event before it returns. If there is no event available on the event list, the method has no effect and returns immediately. 

To determine the time of the next event, we can use the simulator's `peek()` function. We can also list all future events on the event list using the simulator's `show_calendar()` function. This function is supposed to be used only for debugging purposes. Listing all events on an event list can be quite expensive for some simulators.

The following example illustrates some of the functions we just metioned in this section.

In [5]:
# %load "../examples/basics/twosims.py"
import simulus

def handle(sim):
    print("'%s' handles event at time %g" % (sim.name, sim.now))

sim1 = simulus.simulator(name="sim1", init_time=100)
sim2 = simulus.simulator(name="sim2", init_time=-100)

for i in range(5, 100, 20):
    sim1.sched(handle, sim1, offset=i)
for i in range(5, 200, 30):
    sim2.sched(handle, sim2, offset=i)
sim1.show_calendar()
sim2.show_calendar()

while True:
    t1, t2 = sim1.peek(), sim2.peek()
    if t1 < simulus.infinite_time:
        sim1.step()
    if t2 < simulus.infinite_time:
        sim2.step()
    if t1 == simulus.infinite_time and \
       t2 == simulus.infinite_time:
        break


list of all future events (num=5) at time 100 on simulator sim1:
  105: dir_evt=handle() 
  125: dir_evt=handle() 
  145: dir_evt=handle() 
  165: dir_evt=handle() 
  185: dir_evt=handle() 
list of all future events (num=7) at time -100 on simulator sim2:
  -95: dir_evt=handle() 
  -65: dir_evt=handle() 
  -35: dir_evt=handle() 
  -5: dir_evt=handle() 
  25: dir_evt=handle() 
  55: dir_evt=handle() 
  85: dir_evt=handle() 
'sim1' handles event at time 105
'sim2' handles event at time -95
'sim1' handles event at time 125
'sim2' handles event at time -65
'sim1' handles event at time 145
'sim2' handles event at time -35
'sim1' handles event at time 165
'sim2' handles event at time -5
'sim1' handles event at time 185
'sim2' handles event at time 25
'sim2' handles event at time 55
'sim2' handles event at time 85


In this example, we create two simulators, the first starting from time 100 and the second from time -100. (Yes, we can use negative simulation time!) For the first simulator, we schedule a function to be invoked for 5 times at a regular time interval of 20 starting at an offset of 5 from the simulator's start time. For the second simulator, we schedule the same function to be invoked for 7 times at a regular time interval of 30 starting at an offset of 5 from that simulator's start time. We step through the scheduled events of the two simulators, processing one event at a time for each simulator in an alterative fashion as long as the simulation has events.

### 2.2  Process Scheduling

Another way to model the world is to use simulation processes. Conceptually, a process is just a thread of control, similar to running through a sequential prgram that constitutes a bunch of statements, if-else branches, while-loops, and function calls. During its execution, a simulation process can be suspended, either sleeping for some time, or being blocked when requesting for some resources currently unavailable. The process will resume execution after the specified time has passed or when the resource blocking conditions have been removed. 

#### 2.2.1  Hello-World Using Process

The following is a modified hello-world example, which we now use a process.

In [6]:
# %load "../examples/basics/helloworld-proc.py"
import simulus

def print_message():
    for _ in range(10):
        print("Hello world at time", sim.now)
        sim.sleep(1)
    
sim = simulus.simulator()
sim.process(print_message, until=10)
sim.run()


Hello world at time 10
Hello world at time 11
Hello world at time 12
Hello world at time 13
Hello world at time 14
Hello world at time 15
Hello world at time 16
Hello world at time 17
Hello world at time 18
Hello world at time 19


In this example, the function `print_message()` is the starting function of a process. We create the process using the `process()` function of the simulator. We schedule the process to run at time 10 using the `until` argument. If the argument is missing, the process will start immediately at the same simulation time.

As with the original hello-world example that uses the `sched()` method, we can pass aguments to the starting function when we create the process---either as positional arguments, or keyworded arguments, or both. They will be passed to the starting function when the process starts running at the designated time.

In this modified hello-world example, the process loops for 10 times; at each iteration, it prints out a hello message and sleeps for 1 second. The `sleep()` method takes one argument: either an `offset` argument, which specifies the amount of time the process will be put on hold, or an `until` argument, which provides the absolute simulation time at which the process should resume execution. We didn't explicitly name the argument as `offset` as in this example; it's the default positional argument.

#### 2.2.2  The Professor's Life as a Process

We now return to the previous example and use a process to simulate the professor's complicated life.

In [7]:
# %load "../examples/basics/professor-proc.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def prof_life():
    while True:
        start_of_the_day = sim.now
        
        sim.sleep(4*3600) # 4 hours from midnight
        print("professor wakes up at "+strnow())
        
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts drinking coffee at "+strnow())
    
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts reading at "+strnow())

        sim.sleep(offset=(15-5)*60) # 15 minus 5 minutes from now
        print("professor finishes drinking coffee at "+strnow())

        sim.sleep(offset=2*3600-10*60) # 2 hours minus 10 minutes from now
        print("professor finishes reading at "+strnow())

        sim.sleep(until=start_of_the_day+6*3600+30*60) # 6:30
        print("professor breakfasts at "+strnow())
        
        sim.sleep(until=start_of_the_day+6*3600+50*60) # 6:50
        print("professor showers at "+strnow())

        sim.sleep(until=start_of_the_day+7*3600+30*60) # 7:30
        print("professor leaves home and drives to school at "+strnow())

        if sim.now < 24*3600:
            # traffic jam at the first day
            sim.sleep(offset=2*3600+45*60) # 2 hours and 45 minutes from now
            print("professor arrives at school at "+strnow())

            if sim.now < 9*3600:
                # if arrives before the 9 o'clock, attend both meetings
                sim.sleep(until=9*3600)
                print("professor has first meeting at "+strnow())

                sim.sleep(until=10*3600)
                print("professor has second meeting at "+strnow())
            else:
                # if late, no the first meeting and resched the second
                sim.sleep(until=11*3600)
                print("professor has second meeting at "+strnow())
        else:
            # no traffic jam in the following days
            sim.sleep(offset=45*60) # 45 minutes from now
            print("professor arrives at school at "+strnow())

        # the rest of the day is a blur for the professor
        rest_of_the_day(start_of_the_day)

def rest_of_the_day(start):
    # sleep until the start of the next day
    sim.sleep(until=start+24*3600)
            
sim = simulus.simulator()
sim.process(prof_life) 
sim.run(until=72*3600)


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

Rather than having many event handlers as in the earlier example, the professor's life now becomes one process. It's one big loop. Each iteration of the loop represents one day of the professor's life. As a process, the professor calls `sleep()` to advance the simulation time. 

It is important to know that the `sleep()` method can only be called when a process is running, either in the starting function of the process, or in a function that's directly or indirectly invoked by the starting function. Otherwise, simulus will raise an exception.

At each iteration, the professor goes through all the chores of the day. At the end of those chores, she calls the function `rest_of_the_day()` to simulate all the activities she would take for the rest of the day. We are still in the process when that function is called. In the function, the process sleeps until the beginning of the next day. After the function returns, the loop continues to the next iteration to simulate the next day. 

#### 2.2.3  Processes Everywhere

Of course, the world does not have just the professor in it. There are other professors and students, and many other people as well. If we simulate a multi-agent system, for example, each agent can be treated an automic entity with its own agenda. So it's natural to simulate such a system using the processes to represent independent agents. Each agent may also use multiple processes. Recall that professor in the previous example actually has coffee and reads at the same time? Conceptually any two overlapping activities can be modeled as separate processes.

Our world is a complicated world. And processes interact. The agents in a multi-agent system may communicate and synchronize with one another and they compete for resources. Simulus provides the necessary facilities for processes to fulfill these functions. We will deal with the more complicated issues later.

As for now, let's examine a simple example showing the creation and execution of multiple processes. We also show a simple way to synchronize the processes by having one process waiting for the completion of other processes.

In [8]:
# %load "../examples/basics/homework.py"
import simulus

from random import seed, expovariate, gauss
seed(12345)

def student(student_id):
    print("student %d starts to work at %g" % 
          (student_id, sim.now))
    sim.sleep(gauss(30, 5)) # work on part one
    sim.sleep(expovariate(1/10.)) # take some rest
    sim.sleep(gauss(60, 10)) # work on part two
    print("student %d finishes working at %g" % 
          (student_id, sim.now))

def homework():
    print("homework assigned at "+str(sim.now))
    # five students working on the homework each starts at a random 
    # time (exponentially distributed with mean of 10)
    students = []
    for i in range(5):
        s = sim.process(student, i, offset=expovariate(1/10.))
        students.append(s)
    # wait for all student processes to complete
    sim.wait(students)
    print("last student finishes homework at "+str(sim.now))

sim = simulus.simulator()
sim.process(homework, offset=10) 
sim.run()


homework assigned at 10
student 1 starts to work at 10.1022
student 3 starts to work at 13.5473
student 4 starts to work at 14.5952
student 0 starts to work at 15.3892
student 2 starts to work at 27.4415
student 4 finishes working at 108.348
student 1 finishes working at 109.514
student 2 finishes working at 111.156
student 0 finishes working at 122.177
student 3 finishes working at 141.183
last student finishes homework at 141.18344556219594


In this example, a `homework` process is created and scheduled to run at time 10, at which the homework is said to be officially assigned. Five students, each represented as a separate process created by the `homework` process, will work on the assignment. Each `student` process starts at a random time, sleeps for some random time to represent the student's working on the first part of the homework, sleeps again for some random time representing the student's taking a break, and finally sleeps for another random time representing the student's working on the second part of the homework. The `homework` process waits for all the student processes to finish using the magical `wait()` method. We will have much to say about the magic behind this method later.

This example is the first time in this tutorial we deal with random numbers. It needs a bit explanation. We use Python's `random` package. In particular, we use the `seed()` function to select a random seed for the default random number generator, so that every time we run this example, we should get the same results. In simulation, we call the series of numbers generated by the `random` package *pseudo-random numbers*. The numbers indeed look random, but they are not really random. They follow a fixed sequence although seemingly unpredictabe. As long as we start the random sequence with a fixed seed, the sequence is guaranteed be the same.

In this example, we use random numbers from two distributions. The `expovariate()` method generates an exponentially distributed random number. The argument to the method is one divided by the desired mean of the distribution. The `gauss()` method generates a Gaussian (normally) distributed random number. The arguments to the method is the desired mean and standard deviation of the normal distribution. 

## 3.  Inter-Process Communication

Processes often need to interact with one another in order to accomplish tasks. In the previous homework example, we saw one simplest way to communicate among the processes: the `homework` process waits for the completion of all `student` processes. Simulus provides a rich set of mechanisms to support inter-process communication. We start by discussing the two basic primitive methods designed specifically for synchronizing and communicating among the simulation processes: one is called a "trap" and the other is called a "semaphore".

### 3.1  Traps

Traps are one-time signaling mechanisms for inter-process communication. A trap has three states. It's "unset" when the trap is first created and and nothing has happended to it. It's "set" when one or more processes are waiting for the trap to be triggered. It turns to "sprung" when the trap has been triggered, after which there will be no more processes waiting for the trap.

The life of a trap is as follows. A trap starts with the "unset" state when it's created. When a process waits for a trap, the trap goes to "set", at which state more processes may come and wait on the same trap, and the trap would remain in the same "set" state. When a process triggers the trap and if the trap is in the "set" state, *all* processes waiting on the trap will be unblocked and resume execution (it's guaranteed there is at least one waiting process when the trap is in the "set" state). The trap will then be transitioned into the "sprung" state. When a process triggers the trap which is in the "unset" state, the trap will just be transitioned to the "sprung" state (since there are no processes currently waiting on the trap). If a trap has "sprung", further waiting on the trap will be considered as an no-op; that is, the processes trying to wait on a "sprung" trap will have not effect; the process will not be suspended. 

#### 3.1.1  One-Time Signaling

A trap is a very simple signaling mechanism. One or more processes can wait on a trap. When another process triggers the trap, all the waiting processes will be released at once. It is important to know that a trap is for **one-time use** only. Once sprung, a trap cannot be triggered any more, or simulus will raise an exception. That is, if a process wants to send multiple signals to other processes, one would have to use multiple traps (or some other synchronization mechanisms as we will discuss later).

The following example shows a simple use case for traps.

In [9]:
# %load "../examples/basics/onetrap.py"
import simulus

def p(idx):
    if idx > 0:
        print("p%d starts at %g and waits on trap" % (idx, sim.now))
        t.wait()
        print("p%d resumes at %g" % (idx, sim.now))
    else:
        print("p%d starts at %g" % (idx, sim.now))
        sim.sleep(5)
        print("p%d triggers the trap at %g" % (idx, sim.now))
        t.trigger()

sim = simulus.simulator()
t = sim.trap()
for i in range(10):
    sim.process(p, i, offset=10+i)
sim.run()


p0 starts at 10
p1 starts at 11 and waits on trap
p2 starts at 12 and waits on trap
p3 starts at 13 and waits on trap
p4 starts at 14 and waits on trap
p5 starts at 15 and waits on trap
p0 triggers the trap at 15
p1 resumes at 15
p2 resumes at 15
p3 resumes at 15
p4 resumes at 15
p5 resumes at 15
p6 starts at 16 and waits on trap
p6 resumes at 16
p7 starts at 17 and waits on trap
p7 resumes at 17
p8 starts at 18 and waits on trap
p8 resumes at 18
p9 starts at 19 and waits on trap
p9 resumes at 19


In this example, we create a trap using the simulator's `trap()` method. We create 10 processes, p0, p1, ... p9. We stagger them to start from time 10 to 19.  Process p0 acts differently from the others: it sleeps for 5 and then triggers the trap. All the other processes, p1 to p9, simply wait on the trap. You can inspect the results from running this example to see whether the processes behave as what you would expect.

#### 3.1.2  Barriers

A barrier can be used to synchronize a group of processes. A barrier means that the processes must stop at the barrier and cannot be allowed to proceed until all processes from the group reach the barrier. When the last process reaches the barrier, all processes can resume execution and continue from the barrier.

It is rather straightforward to implement barriers using traps. In the following example, we creates such a barrier.

In [10]:
# %load "../examples/basics/barrier.py"
import simulus

from random import seed, expovariate
seed(12345)

class Barrier(object):
    def __init__(self, sim, total_procs):
        self.sim = sim
        self.total_procs = total_procs
        # reset the barrier (by creating a new trap and 
        # resetting the count)
        self.trap = self.sim.trap()
        self.num = 0
        
    def barrier(self):
        self.num += 1
        if self.num < self.total_procs:
            # not yet all processes have reached the barrier
            self.trap.wait()
        else:
            # the last process has reached the barrier
            self.trap.trigger()
            # reset the barrier for next time use (by 
            # creating a new trap and resetting the count)
            self.trap = self.sim.trap()
            self.num = 0

def p(idx):
    while True:
        t = expovariate(1)
        print("p%d runs at %g and sleeps for %g" % (idx, sim.now, t))
        sim.sleep(t)
        print("p%d reaches barrier at %g" % (idx, sim.now))
        bar.barrier()
        
sim = simulus.simulator()
bar = Barrier(sim, 10)
for i in range(10):
    sim.process(p, i)
sim.run(10)


p0 runs at 0 and sleeps for 0.538916
p2 runs at 0 and sleeps for 0.0102212
p6 runs at 0 and sleeps for 1.74415
p9 runs at 0 and sleeps for 0.354734
p5 runs at 0 and sleeps for 0.459518
p1 runs at 0 and sleeps for 0.215251
p4 runs at 0 and sleeps for 0.83473
p3 runs at 0 and sleeps for 0.176365
p8 runs at 0 and sleeps for 0.132694
p7 runs at 0 and sleeps for 0.567284
p2 reaches barrier at 0.0102212
p8 reaches barrier at 0.132694
p3 reaches barrier at 0.176365
p1 reaches barrier at 0.215251
p9 reaches barrier at 0.354734
p5 reaches barrier at 0.459518
p0 reaches barrier at 0.538916
p7 reaches barrier at 0.567284
p4 reaches barrier at 0.83473
p6 reaches barrier at 1.74415
p6 runs at 1.74415 and sleeps for 0.825716
p2 runs at 1.74415 and sleeps for 0.191577
p8 runs at 1.74415 and sleeps for 0.805691
p3 runs at 1.74415 and sleeps for 0.438352
p1 runs at 1.74415 and sleeps for 3.17163
p9 runs at 1.74415 and sleeps for 0.0957338
p5 runs at 1.74415 and sleeps for 3.84624
p0 runs at 1.74415 and

The `Barrier` class implements the barrier. One creates a barrier with two arguments: the simulator on which the processes are run and the total number of processes expected at the barrier. Internally, we create a trap for the synchronization and use a counter (called `num`) to keep track of the number of processes having reached the barrier so far. The counter is initialized to zero. Each time a process wants to use the barrier, it calls the `barrier()` method, which increments the counter. If the counter is smaller than the total number of processes, the process will be put on hold (using the trap's `wait()` method). Otherwise, if the counter gets to the total number of processes, the process is the last one among the group of processes to reach the barrier. Therefore, it calls the trap's `trigger()` method to release all the waiting processes that have earlier arrived at the barrier. Remember that traps can only be used once. To make the barrier reusable, we create a new trap each time the last process reaches the barrier. We also reset the counter.

In this example, we create 10 processes. Each process waits for some random time (exponentially distributed) and then calls `barrier()`. Once the method returns, the process repeats the same: it waits for some random time and calls for the next barrier.

### 3.2  Semaphores

Semaphores are multi-use signaling mechanisms for inter-process communication. It is the other primitive method beside traps designed for synchronizing and communicating simulation processes. 

In simulus, a semaphore implements what is commonly called a "counting semaphore." Initially, a semaphore can have a nonnegative integer count, which indicates the number of available resources. The processes atomically increment the semaphore count to represent resources being added or returned to the pool (using the `signal()` method). Similarly, the processes atomically decrement the semaphore count to represent resources being removed from the pool (using the `wait()` method). When the semaphore count is zero, it means that there are no available resources. In that case, a process trying to decrement the semaphore (to remove a resource) will be blocked until more resources are added back to the pool. 

A semaphore is different than a trap. A trap is a one-time signaling mechanism. Multiple processes can wait on a trap. Once a process triggers the trap, *all* waiting processes will be unblocked. Moreover, a trap cannot be reused: once a trap is sprung, subsequent waits will not block the processes and a trap cannot be triggered again. In comparison, a semaphore is a multi-use signaling mechanism. Each time a process waits on a semaphore, the semaphore value will be decremented. If the value becomes negative, the process will be blocked. Each time one signals a semaphore, the semaphore value will be incremented. If there are blocked processes, *one* of these processes will be unblocked. Processes can use the same semaphore continuously multiple times.

#### 3.2.1 Circular Wait

The following example shows the use of semaphores to synchronize a group of processes.  

In [11]:
# %load "../examples/basics/circular.py"
import simulus

from random import seed, expovariate
seed(12345)

def p(idx):
    while True:
        sems[idx].wait()
        print("p%d wakes up at %g" % (idx, sim.now))
        sim.sleep(expovariate(1))
        sems[(idx+1)%total_procs].signal()
        
sim = simulus.simulator()

total_procs = 10
sems = [sim.semaphore() for _ in range(total_procs)]
for i in range(10):
    sim.process(p, i)
sems[0].signal()
sim.run(20)


p0 wakes up at 0
p1 wakes up at 0.538916
p2 wakes up at 0.549138
p3 wakes up at 2.29329
p4 wakes up at 2.64802
p5 wakes up at 3.10754
p6 wakes up at 3.32279
p7 wakes up at 4.15752
p8 wakes up at 4.33388
p9 wakes up at 4.46658
p0 wakes up at 5.03386
p1 wakes up at 5.85958
p2 wakes up at 6.05115
p3 wakes up at 6.85685
p4 wakes up at 7.2952
p5 wakes up at 10.4668
p6 wakes up at 10.5626
p7 wakes up at 14.4088
p8 wakes up at 14.94
p9 wakes up at 15.6411
p0 wakes up at 15.8014
p1 wakes up at 17.0707
p2 wakes up at 17.2814
p3 wakes up at 17.6993
p4 wakes up at 17.7231
p5 wakes up at 18.1379


In this example, ten processes are organized in a circle. Each process is created with a semaphore; it waits for the semaphore and then signal the semaphore of the subsequent process, and then repeats. In this case, all the processes are executed one at a time in a round robin fashion.

The use of semaphores in this case is very much like traps, since there is at most one process waiting on a semaphore at any given time. Whether the semaphore unblocks just one waiting process at a time, or the trap unblocks all the waiting processes at once does not really matter in this case. However, a semaphore can be reused, while a trap cannot. If we use traps for this example, we would have to create a new trap each time a process wakes up (similar to what we did for the barrier example).

#### 3.2.2  Producer-Consumer Problem

We should have learned from our Operation Systems class about the producer–consumer problem (also known as the bounded-buffer problem). It's a classic scenario for multi-process synchronization. In its simpliest form, the problem consists of two processes. A producer process repeatedly generates data and put them into a common, fixed sized buffer. A consumer consumes the data by removing them from the buffer one at a time. The problem is that the producer cannot put data into the buffer if the buffer is already full. Similarly, the consumer cannot remove data from the buffer if the buffer is empty.

One solution to the producer-consumer problem is to use semaphores, as shown in the following example. Later we will show an even simpler way to solve this problem (using `store` provided by simulus).

In [12]:
# %load "../examples/basics/boundbuf.py"
import simulus

from random import seed, expovariate, gauss
seed(12345)

bufsiz = 5 # buffer capacity
items_produced = 0 # keep track the number of items produced
items_consumed = 0 # ... and consumed for ALL producers and consumers
num_producers = 2 # number of producers
num_consumers = 3 # number of consumers

def producer(idx):
    global items_produced
    while True:
        sim.sleep(expovariate(1)) # take time to produce an item
        num = items_produced
        items_produced += 1
        print("%f: p[%d] produces item [%d]" % (sim.now, idx, num))
        sem_avail.wait() # require a free slot in buffer
        sem_occupy.signal() # store the item and increase occupancy
        print("%f: p[%d] stores item [%d] in buffer" % 
              (sim.now, idx, num))
        
def consumer(idx):
    global items_consumed
    while True:
        sem_occupy.wait() # require an item from buffer
        sem_avail.signal() # retrieve the item and bump the free slots
        num = items_consumed
        items_consumed += 1
        print("%f: c[%d] retrieves item [%d] from buffer" %
              (sim.now, idx, num))
        sim.sleep(gauss(0.8, 0.2)) # take time to consume the item
        print("%f: c[%d] consumes item[%d]" % (sim.now, idx, num))        

sim = simulus.simulator()
sem_avail = sim.semaphore(bufsiz) # available slots
sem_occupy = sim.semaphore(0) # no items yet
for i in range(num_producers): 
    sim.process(producer, i)
for i in range(num_consumers):
    sim.process(consumer, i)
sim.run(10)


0.010221: p[1] produces item [0]
0.010221: p[1] stores item [0] in buffer
0.010221: c[0] retrieves item [0] from buffer
0.538916: p[0] produces item [1]
0.538916: p[0] stores item [1] in buffer
0.538916: c[2] retrieves item [1] from buffer
0.752533: c[0] consumes item[0]
0.754168: p[0] produces item [2]
0.754168: p[0] stores item [2] in buffer
0.754168: c[1] retrieves item [2] from buffer
1.521765: c[2] consumes item[1]
1.588897: p[0] produces item [3]
1.588897: p[0] stores item [3] in buffer
1.588897: c[0] retrieves item [3] from buffer
1.608449: c[1] consumes item[2]
1.754371: p[1] produces item [4]
1.754371: p[1] stores item [4] in buffer
1.754371: c[2] retrieves item [4] from buffer
2.156181: p[0] produces item [5]
2.156181: p[0] stores item [5] in buffer
2.156181: c[1] retrieves item [5] from buffer
2.476470: c[0] consumes item[3]
2.580087: p[1] produces item [6]
2.580087: p[1] stores item [6] in buffer
2.580087: c[0] retrieves item [6] from buffer
2.594533: p[0] produces item [7]

We use two semaphores: one semaphore `sem_avail` is used to count the number of free slots in the buffer, and the other semaphore `sem_occupy` is used to count the number of produced items stored in the buffer. 

A producer sleeps for some random time which is exponentially distributed to represent the production of an item, then calls `wait()` on the semaphore `sem_avail` to decrement the available slots in the buffer. The process may be blocked if there is no more free slots available, in which case we wait for a consumer to retrieve an item and thereby creates a free slot. If the buffer is not full, the producer adds the item into the buffer, which is represented by calling `signal()` on the semaphore `sem_occupy`, which increments the number of occupied items in the buffer (which could potential unblock a waiting consumer process). The producer then repeats.

A consumer calls `wait()` on the semaphore `sem_occupy` to decrement the number of items stored in the buffer. The process may be blocked if there are currently no items in the buffer. In this case, the process will wait until a producer adds an item. Otherwise, the consumer retrieves the item, which is represented by calling `signal()` on the semaphore `sem_avail`, which increments the number of available slots in the buffer. This could potentially unblock a waiting producer process. The consume process then consumes the item, by randomly waiting for some random time, which is normally distributed. The consumer then repeats.

In this example, we create two producer and three consumer processes, and use the two semaphores to synchronize the processes to access the bounded buffer. If you're familiar with the performance modeling literature, you may recognize that we are actually simulating a multi-server queue with limited capacity.

#### 3.2.3  Queuing Disciplines

If multiple processes are waiting on a semaphore, the order in which the processes are unblocked may be important. By default, a semaphore applies the FIFO order (first in first out). That is, the first process which got blocked on the semaphore will be the first one unblocked. 

Other queuing disciplines are also possible, including LIFO (last in first out), RANDOM, and PRIORITY (which is based on the 'priority' of the processes: a lower value means higher priority). One can choose a queuing discipline when the semaphore is created. The queuing disciplines are constants defined in the `QDIS` class.

In the following example, we show the use of different queuing disciplines.

In [13]:
# %load "../examples/basics/qdis.py"
import simulus

# so that we get same result from random priority
from random import seed
seed(12345)

def p(idx, sem):
    # set the priority of the current process (this is only useful 
    # if we use PRIORITY qdis)
    sim.set_priority(abs(idx-3.2))

    # make sure the process wait on the semaphore in order
    sim.sleep(idx)

    # the process will block on the semaphore and then print out 
    # a message when it is unblocked
    sem.wait()
    print("p[id=%d,prio=%.1f] resumes at %f" % 
          (idx, sim.get_priority(), sim.now))

def trywaits(sem):
    # create ten processes which will all block on the semaphore
    for i in range(10):
        sim.process(p, idx=i, sem=sem)
    sim.sleep(10)
    
    # release them all and check the order they are unblocked
    print('-'*40)
    for i in range(10):
        sem.signal()

sim = simulus.simulator()
s1 = sim.semaphore()
s2 = sim.semaphore(qdis=simulus.QDIS.LIFO)
s3 = sim.semaphore(qdis=simulus.QDIS.RANDOM)
s4 = sim.semaphore(qdis=simulus.QDIS.PRIORITY)
sim.process(trywaits, s1, offset=0)
sim.process(trywaits, s2, offset=100)
sim.process(trywaits, s3, offset=200)
sim.process(trywaits, s4, offset=300)
sim.run()


----------------------------------------
p[id=0,prio=3.2] resumes at 10.000000
p[id=1,prio=2.2] resumes at 10.000000
p[id=2,prio=1.2] resumes at 10.000000
p[id=3,prio=0.2] resumes at 10.000000
p[id=4,prio=0.8] resumes at 10.000000
p[id=5,prio=1.8] resumes at 10.000000
p[id=6,prio=2.8] resumes at 10.000000
p[id=7,prio=3.8] resumes at 10.000000
p[id=8,prio=4.8] resumes at 10.000000
p[id=9,prio=5.8] resumes at 10.000000
----------------------------------------
p[id=9,prio=5.8] resumes at 110.000000
p[id=8,prio=4.8] resumes at 110.000000
p[id=7,prio=3.8] resumes at 110.000000
p[id=6,prio=2.8] resumes at 110.000000
p[id=5,prio=1.8] resumes at 110.000000
p[id=4,prio=0.8] resumes at 110.000000
p[id=3,prio=0.2] resumes at 110.000000
p[id=2,prio=1.2] resumes at 110.000000
p[id=1,prio=2.2] resumes at 110.000000
p[id=0,prio=3.2] resumes at 110.000000
----------------------------------------
p[id=5,prio=1.8] resumes at 210.000000
p[id=1,prio=2.2] resumes at 210.000000
p[id=2,prio=1.2] resumes at 2

We create four semaphores, each selected for a different queuing discipline. This is achieved by passing the `qdis` argument to the simulator's `semaphore()` method that creates the semaphores. To use the priority based queuing discipline, we use the simulator's `set_priority()` method to set the current process's priority. 

For each of the four semaphores, we create a `trywaits` process, which then creates another 10 processes who will wait on the semaphore. Then the `trywaits` process will release all the waiting processes one by one. From the print-out, we should be able to determine the order in which the waiting processes are unblocked. 

### 3.3  Trappables and Conditional Wait

Both traps and semaphores are called **trappables**. An event (i.e., a scheduled function) is a trappable. A process is also a trappable. Simulus provides a very powerful function called `wait()` to block the calling process until any or all of the given trappables are triggered, or until a pre-specified amount of time has elapsed. When we say a trappable is "triggered", we mean that the wait condition on the trappable has been satisfied. If it's a trap, it means the trap has been triggered by another process. If it's a semaphore, it means a wait on the semaphore has returned. If it's an event, it means the event has happened (and the event handler has been invoked). If it's a process, it means the process has terminated. 

In case of a trap, if we have only one trap, say `t`, calling the wait method of the trap, `t.wait()`, is equivalent to calling the simulator's `wait()` function with the trap passed as the argument: `sim.wait(t)`. Similarly, in case of a semaphore, if we have only one semaphore, say `s`, `s.wait()` is equivalent to `sim.wait(s)`. 

The simulator's `wait()` function expects the argument to be either one trappable or a list of trappables. If it takes more than one trappables in a list or tuple, the calling process will be blocked until either *one* of the trappables, or *all* of the trappables are triggered. The behavior depends on the `method` argument: if it's `any`, the wait condition is satisfied as soon as one of the trappables is triggered; if it's 'all', the process needs to wait until all trappables are triggered. If the method argument is not provided, simulus assumes it's 'all' by default.

#### 3.3.1  Waiting on Multiple Trappables

The following shows an example of a process waiting for multiple trappables.

In [14]:
# %load "../examples/basics/multiwait.py"
import simulus

def p1():
    sim.sleep(10)
    print("p1 triggers t1 at %g" % sim.now)
    t1.trigger()

    sim.sleep(10)
    print("p1 triggers s1 at %g" % sim.now)
    s1.trigger() # signal and trigger are aliases for semaphore

    sim.sleep(10)
    print("p1 triggers t2 at %g" % sim.now)
    t2.trigger()

    sim.sleep(10)
    print("p1 triggers s2 at %g" % sim.now)
    s2.signal()

def p2():
    tp = (t1, s1)
    r = sim.wait(tp)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

    tp = [t2, s2]
    r = sim.wait(tp, method=any)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

    # find the remaining untriggered trappables (using the 
    # returned mask) and wait for them all
    tp = [t for i, t in enumerate(tp) if not r[i]]
    r = sim.wait(tp)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

sim = simulus.simulator()
t1 = sim.trap()
t2 = sim.trap()
s1 = sim.semaphore()
s2 = sim.semaphore()
sim.process(p1)
sim.process(p2)
sim.run()


p1 triggers t1 at 10
p1 triggers s1 at 20
p2 resumes at 20 (ret=([True, True], False))
p1 triggers t2 at 30
p2 resumes at 30 (ret=([True, False], False))
p1 triggers s2 at 40
p2 resumes at 40 (ret=([True], False))


In this example, we create two traps, t1 and t2, and two semaphores, s1 and s2. We also create two processes. The process p1 triggers a trap or a semaphore at 10 seconds interval. The other process p2 first waits on a trap and a semaphore using the default method ('all'). As expected, the process only resumes execution once both trappables have been triggered. The p2 process then waits on the other trap and semaphore using the `any` method. In this case, when one trappable is triggered, the process will resume execution. It then filters out the triggered trappables from the list and creates another list with the remaining untriggered trappables (actually there's only one left), and then wait until all of the remaining trapples are triggered.

The return from the `wait()` function needs a bit more explanation. The `wait()` function actually returns a tuple with two elements: the first element of the tuple indicates whether the trappables have been triggered or not; and the second element of tuple indicates whether timeout happens. Since in this example, we don't use timed wait, the `wait()` function always returns False in the second element.

If the first argument when calling the `wait()` function is but one trappable (not in a list or tuple), the first element of the returned tuple will simply be a boolean (True or False), indicating whether the trappable has been triggered or not upon the return of the function. If, on the other hand, the first argument when calling the `wait()` functionis a list or a tuple of trappables (even if with just one trappable), the first element of the returned tuple will be a list of booleans, where each element of the list indicates whether the corresponding trappable has been triggered.

In the example, when the `wait()` function is called the first time, it returns a list with True and True for the first element of the returned tuple, since both trappables must be triggered before the process can resume execution (because of the 'all' method). In the second time, the function returns True and False. Only the first trappable (the trap t2) is triggered at the time. Because of the method is 'any', one triggered trappable is good enough to satisfy the wait condition. at the third time, the remaining trappable (semaphore s2) is triggered and the function returns a list consisted of only one True element.

We can optionally provide an 'offset' or an 'until' argument to the `wait()` function. The 'offset' is the relative time from now until which the process will be put on hold at the latest; if provided, it must be a non-negative value. The 'until' is the absolute time at which the process is expected to resume execution at the latest; if provided, it must not be earlier than the current time.  Eœither 'offset' or 'until' can be provided, but not both. If both 'offset' and 'until' are ignored (like what we have previously), there will be no time limit on the conditional wait.

#### 3.3.2  Race between Tom and Jerry

To be able to perform conditional wait on multiple trappables certainly allows a process-oriented model to be quite expressive. The following shows a good example. Tom and Jerry decides to enter a race. Tom is modeled by processes. Each time Tom enters the race, we create a process, which calls `sleep()` to represent the time duration for the run. The time duration is a random variable from a normal distribution with a mean of 100 and a standard deviation of 50 (and a cutoff below zero). Jerry is modeled by events. Each time Jerry enters the race, we schedule an event using `sched()` with a time offset representing the time duration for the run. The time duration is a random variable from a uniform distribution between 50 and 100. Tom and Jerry compete for ten times; the next race would start as soon as the previous one finishes. For each race, whoever runs the fastest wins. But if they run for more than 100, both are disqualified for that race. The simulation finds out who eventually wins more races.

In [15]:
# %load "../examples/basics/tomjerry.py"
import simulus

from random import seed, gauss, uniform
seed(321)

def tom():
    sim.sleep(max(0, gauss(100, 50)))
    print("%g: tom finished" % sim.now)

def jerry():
    print("%g: jerry finished" % sim.now)

def compete():
    tom_won, jerry_won = 0, 0
    for _ in range(10):
        print("<-- competition starts at %g -->" % sim.now)

        p = sim.process(tom) # run, tom, run!
        e = sim.sched(jerry, offset=uniform(50, 150)) # run, jerry, run!
    
        # let the race begin...
        (r1, r2), timedout = sim.wait((p, e), 100, method=any)
        if timedout:
            print("%g: both disqualified" % sim.now)
            sim.cancel(p)
            sim.cancel(e)
        elif r1: 
            print("%g: tom wins" % sim.now)
            tom_won += 1
            sim.cancel(e)
        else:
            print("%g: jerry wins" % sim.now)
            jerry_won += 1
            sim.cancel(p)
    print("final result: tom:jerry=%d:%d" % (tom_won, jerry_won))
    
sim = simulus.simulator()
sim.process(compete)
sim.run()


<-- competition starts at 0 -->
77.5459: jerry finished
77.5459: jerry wins
<-- competition starts at 77.5459 -->
171.749: jerry finished
171.749: jerry wins
<-- competition starts at 171.749 -->
271.749: both disqualified
<-- competition starts at 271.749 -->
357.072: tom finished
357.072: tom wins
<-- competition starts at 357.072 -->
430.387: tom finished
430.387: tom wins
<-- competition starts at 430.387 -->
485.297: tom finished
485.297: tom wins
<-- competition starts at 485.297 -->
585.297: both disqualified
<-- competition starts at 585.297 -->
611.838: tom finished
611.838: tom wins
<-- competition starts at 611.838 -->
711.838: both disqualified
<-- competition starts at 711.838 -->
811.838: both disqualified
final result: tom:jerry=4:2


Earlier in the previous example, we showed traps and semaphores as trappables on which we can perform conditional wait. In this example, we show that both events and processes are also trappables. An instance of the process is returned from the `process()` function, and an instance of the event is returned from the `sched()` function. Both are considered as opaque objects. That is, the users are not expected to inspect the content of a process or event. Rather, the users should simple use the references. A process is a trappable, which is triggered when the process is terminated. An event is also a trappable, which is triggered upon the activation of the event (i.e., when the event handler is invoked).

In this example, we call the simulator's `wait()` function to wait for 'any' of the trappables to be triggered. We also provide a time limit of 100 (we use 'offset' as a positional argument). We look at the return value from the `wait()` function to determine the outcome of the race. If the wait is timed out, we need to kill the process and cancel the event. This is done by calling the simulator's `cancel()` method. If Tom wins (when p is triggered), we just cancel the event; and if Jerry wins (when e is triggered), we instead kill the process.

## 4. Resources and Facilities

In the previous section, we discussed traps and semaphores, which are the two primitive methods in simulus for process synchronization. Simulus provides several advanced mechanisms to coordinate processes and facilitate inter-process communication in order to ease the modeling effort. In this section, we discuss three such mechanisms: resource, store, and mailbox.

### 4.1 Resource

A semaphore is a primitive method to represent a resource. A process can atomically increment the semaphore to represent a resource has been added to the pool. A process can also atomically decrement the semaphore to represent a resource being removed from the pool. 

Simulus actually provides a high-level abstraction of the resources, making it easier for users to create expressive models. A resource basically models a single-server or multi-server queue. A resource can allow only a limited number of processes to be serviced at any given time. A process arrives and acquires a server at the resource. If there is an available server, the process will gain access to the resource for as long as the service is required. If there isn't an available server, the process will be put on hold and placed in a waiting queue. When another process has finished and released the server, one of the waiting processes will be unblocked and thereby gain access to the resource.

A process is expected to perform the following sequence of actions to use the resource. The process first calls the `acquire()` method to gain access to a server. This is potentially a blocking call: the process may be blocked if there's no avaiable server. The call returns if a server is available and has been assigned to the process. The process has acquired the resource. The process can then use the resource for as long as it needs to; this is usually modeled using the `sleep()` method. Afterwards, the same process is expected to call the `release()` method on the resource to free the server, so that other waiting processes may have a chance to gain access to the resource.

#### 4.1.1 A Single-Server Queue

The following example shows the use of resource for simulating a single-server queue. It's an M/M/1 queue with exponentially distributed inter-arrival time and exponentially distributed service time. We first create a resource with one server (capacity is one by default). We create an arrival process, which creates a job after sleeping for some random inter-arrival time in a forever loop. A job is a process. It first tries to `acquire()` a server from the resource. If the resource is unavailable (no available server), the job process will be suspended. Otherwise, the job is assigned with a server. The process sleeps for some random service time and then calls `release()` to free the server before leaving the queue (and terminating).

In [16]:
# %load "../examples/basics/mm1.py"
import simulus

from random import seed, expovariate
seed(123)

def job(idx):
    r.acquire()
    print("%g: job(%d) gains access " % (sim.now,idx))
    sim.sleep(expovariate(1))
    print("%g: job(%d) releases" % (sim.now,idx))
    r.release()
    
def arrival():
    i = 0
    while True:
        i += 1
        sim.sleep(expovariate(0.95))
        print("%g: job(%d) arrives" % (sim.now,i))
        sim.process(job, i)

sim = simulus.simulator()
r = sim.resource()
sim.process(arrival)
sim.run(10)


0.0566152: job(1) arrives
0.0566152: job(1) gains access 
0.15264: job(2) arrives
0.272591: job(3) arrives
0.579584: job(1) releases
0.579584: job(2) gains access 
0.618484: job(2) releases
0.618484: job(3) gains access 
1.38679: job(3) releases
2.70906: job(4) arrives
2.70906: job(4) gains access 
3.13407: job(5) arrives
3.31718: job(6) arrives
3.75014: job(7) arrives
4.17767: job(8) arrives
4.47373: job(9) arrives
4.47549: job(10) arrives
4.62019: job(4) releases
4.62019: job(5) gains access 
4.71188: job(5) releases
4.71188: job(6) gains access 
5.07885: job(11) arrives
5.1551: job(12) arrives
5.55405: job(13) arrives
5.62219: job(6) releases
5.62219: job(7) gains access 
6.18015: job(14) arrives
6.28263: job(15) arrives
6.44405: job(16) arrives
7.98027: job(7) releases
7.98027: job(8) gains access 
8.00174: job(8) releases
8.00174: job(9) gains access 
8.0872: job(17) arrives
8.98396: job(18) arrives
9.30851: job(19) arrives


#### 4.1.2 Conditional Wait on Resources

A resource is also trappable. That means it can be presented to the simulator's `wait()` function as an argument so that we can apply conditional wait on it. A resource is triggered if the process acquires a server and gains access to the resource. 

We illustrate the use of conditional wait on resources with the bank renege example, which was originally written for the SimPy simulator. This example models a bank where customers arrive randomly. The bank counter can serve only one customer at a time. When a customer arrives and the bank counter is currently occupied, the customer enters a queue. Each customer also has a certain patience. If a customer waits too long, the customer may decide to abort, thus leaving the queue and the bank. (In queuing theory, this is called "reneging"). 


In [17]:
# %load "../examples/simpy/bank.py"
"""This example is modified from the simpy's bank renege example; we
use the same settings as simpy so that we can get the same results."""

RANDOM_SEED = 42        # random seed for repeatability
NUM_CUSTOMERS = 5       # total number of customers
INTV_CUSTOMERS = 10.0   # mean time between new customers
MEAN_BANK_TIME = 12.0   # mean time in bank for each customer
MIN_PATIENCE = 1        # min customer patience
MAX_PATIENCE = 3        # max customer patience

import simulus
from random import seed, expovariate, uniform

def source():
    for i in range(NUM_CUSTOMERS):
        sim.process(customer, i)
        sim.sleep(expovariate(1.0/INTV_CUSTOMERS))

def customer(idx):
    arrive = sim.now
    print('%7.4f Customer%02d: Here I am' % (arrive, idx))

    patience = uniform(MIN_PATIENCE, MAX_PATIENCE)
    _, timedout = sim.wait(counter, patience)
    if timedout:
        print('%7.4f Customer%02d: RENEGED after %6.3f' %
              (sim.now, idx, sim.now-arrive))
    else:
        print('%7.4f Customer%02d: Waited %6.3f' %
              (sim.now, idx, sim.now-arrive))
        sim.sleep(expovariate(1.0/MEAN_BANK_TIME))
        print('%7.4f Customer%02d: Finished' % (sim.now, idx))
        counter.release()

print('Bank renege')
seed(RANDOM_SEED)
sim = simulus.simulator()
counter = sim.resource()
sim.process(source)
sim.run()


Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


### 4.2 Store

A store is used for synchronizing producer and consumer processes. A store is a facility either for storing countable objects (such as jobs in a queue, packets in a network router, and io requests at a storage device), or for storing uncountable quantities or volumes (such as gas in a tank, water in a reservoir, and battery power in a mobile device). The user can determine which kind of store (for countable objects or for uncountable quantities) should apply upon use.

A store has a maximum capacity, which is a positive quantity specified either as an integer or as a float-point number. A store can also tell its current storage level, which goes between zero and the maximum capacity. One or several processes can put objects or quantities into the store. They are called the "producer processes". One or several processes can get objects or quantities from the store. They are called the "consumer processes". The producer processes and the consumer processes are determined by their performed actions on the store. They can actually be the same process.

A producer process calls the `put()` method to deposit one or more objects or some quantities into the store. The put amount shall be specified as an argument (default is one). The current storage level will increase accordingly as a result. However, if a producer process tries to put objects or quantities into the store such that the storage level would be more than the store's capacity, the producer process will be blocked. The process will remain to be blocked until the storage level decreases (when some other processes get objects or quantities from the store) so that there is room for putting all the objects or quantities.

Similarly, a consumer process calls the `get()` method to retrieve one or more objects or some quantities from the store. The get amount shall be specified as an argument (default is one).  The current storage level will decrease accordingly as a result. If a consumer process tries to get more objects or quantities than what is avaialble at the store, the consumer process will be blocked. The process will remain blocked until the current storage level goes above the requested amount (when some other processes put objects or quantities into the store).

By default, we only specify the amount of objects or quantities to put into the store or get from the store. Actually, the store facility can be used for storing real (countable) Python objects, if the user calls the `put()` method and passes in a Python object or a list/tuple of Python objects using the keyworded 'obj' argument. In this case, the put amount must match with the number of objects. These Python objects can be retrieved in a first-in-first-out fashion by consumer processes calling the `get()` method, which specifies the get amount. The same number of Python objects will be returned, either in a list if the get amount is greater than one, or by the object itself if the get amount is one.

#### 4.2.1 Producer-Consumer  Problem Revisited

Earlier we used two semaphores to solve the producer-consumer (bounded buffer) problem. In the following example, we use the store facility to achieve the same goal.

In [18]:
# %load '../examples/basics/boundbuf-store.py'
import simulus

from random import seed, expovariate, gauss
seed(12345)

bufsiz = 5 # buffer capacity
items_produced = 0 # keep track the number of items produced
num_producers = 2 # number of producers
num_consumers = 3 # number of consumers

def producer(idx):
    global items_produced
    while True:
        sim.sleep(expovariate(1)) # take time to produce an item
        num = items_produced
        items_produced += 1
        print("%f: p[%d] produces item [%d]" % (sim.now, idx, num))
        s.put(obj=num)
        print("%f: p[%d] stores item [%d] in buffer" % 
              (sim.now, idx, num))
        
def consumer(idx):
    while True:
        num = s.get()
        print("%f: c[%d] retrieves item [%d] from buffer" %
              (sim.now, idx, num))
        sim.sleep(gauss(0.8, 0.2)) # take time to consume the item
        print("%f: c[%d] consumes item[%d]" % (sim.now, idx, num))        

sim = simulus.simulator()
s = sim.store(capacity=bufsiz)
for i in range(num_producers): 
    sim.process(producer, i)
for i in range(num_consumers):
    sim.process(consumer, i)
sim.run(10)


0.010221: p[1] produces item [0]
0.010221: p[1] stores item [0] in buffer
0.010221: c[0] retrieves item [0] from buffer
0.538916: p[0] produces item [1]
0.538916: p[0] stores item [1] in buffer
0.538916: c[2] retrieves item [1] from buffer
0.752533: c[0] consumes item[0]
0.754168: p[0] produces item [2]
0.754168: p[0] stores item [2] in buffer
0.754168: c[1] retrieves item [2] from buffer
1.521765: c[2] consumes item[1]
1.588897: p[0] produces item [3]
1.588897: p[0] stores item [3] in buffer
1.588897: c[0] retrieves item [3] from buffer
1.608449: c[1] consumes item[2]
1.754371: p[1] produces item [4]
1.754371: p[1] stores item [4] in buffer
1.754371: c[2] retrieves item [4] from buffer
2.156181: p[0] produces item [5]
2.156181: p[0] stores item [5] in buffer
2.156181: c[1] retrieves item [5] from buffer
2.476470: c[0] consumes item[3]
2.580087: p[1] produces item [6]
2.580087: p[1] stores item [6] in buffer
2.580087: c[0] retrieves item [6] from buffer
2.594533: p[0] produces item [7]

In this example, we create a store with capacity being the buffer size. We create two producer processes and three consumer processes. Each producer process repeatedly sleeps for some time and then deposits an item to the store. An integer (the serial number of the created item) is passed in as the real object. Each consumer process first tries to get an item from the store. Since the store uses real objects, an earlier deposited integer will be returned. The consumer process then sleeps for some random time before it repeats in the loop.

#### 4.2.2  Uncountable Quantities

The previous example shows that we can use store to represent the production and consumption of countable objects. Store can also be used to represent the storage of uncountable quantities or volumes (gas, water, and battery power). The following example, which was originally written for the SimPy simulator, models a gas station and cars arriving at the station randomly for refueling.

In [19]:
# %load "../examples/simpy/gas.py"
"""This example is modified from the simpy's gas station refueling
example; we use the same settings as simpy so that we can get the same
results."""

RANDOM_SEED = 42           # random seed for repeatability
GAS_STATION_SIZE = 200     # liters
THRESHOLD = 10             # Threshold for calling the tank truck (in %)
FUEL_TANK_SIZE = 50        # liters
FUEL_TANK_LEVEL = [5, 25]  # Min/max levels of fuel tanks (in liters)
REFUELING_SPEED = 2        # liters / second
TANK_TRUCK_TIME = 300      # Seconds it takes the tank truck to arrive
T_INTER = [30, 300]        # Create a car every [min, max] seconds
SIM_TIME = 1000            # Simulation time in seconds

import random, itertools
import simulus

def car_generator(sim, gas_station, fuel_pump):
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
        sim.sleep(random.randint(*T_INTER))
        sim.process(car, 'Car %d' % i, sim, gas_station, fuel_pump)

def car(name, sim, gas_station, fuel_pump):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get
    the desired amount of gas from it. If the stations reservoir is
    depleted, the car has to wait for the tank truck to arrive.

    """
    
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print('%s arriving at gas station at %.1f' % (name, sim.now))

    start = sim.now
    # Request one of the gas pumps
    gas_station.acquire()

    # Get the required amount of fuel
    liters_required = FUEL_TANK_SIZE - fuel_tank_level
    fuel_pump.get(liters_required)

    # The "actual" refueling process takes some time
    sim.sleep(liters_required / REFUELING_SPEED)

    gas_station.release()
    print('%s finished refueling in %.1f seconds.' %
          (name, sim.now - start))

def gas_station_control(sim, fuel_pump):
    """Periodically check the level of the *fuel_pump* and call the tank
    truck if the level falls below a threshold."""

    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # We need to call the tank truck now!
            print('Calling tank truck at %d' % sim.now)

            # Wait for the tank truck to arrive and refuel the station
            sim.wait(sim.process(tank_truck, sim, fuel_pump))

        sim.sleep(10)  # Check every 10 seconds

def tank_truck(sim, fuel_pump):
    """Arrives at the gas station after a certain delay and refuels it."""
    
    sim.sleep(TANK_TRUCK_TIME)
    print('Tank truck arriving at time %d' % sim.now)
    
    ammount = fuel_pump.capacity - fuel_pump.level
    print('Tank truck refuelling %.1f liters.' % ammount)
    fuel_pump.put(ammount)

# Setup and start the simulation
print('Gas Station refuelling')
random.seed(RANDOM_SEED)

# Create simulator and start processes
sim = simulus.simulator()
gas_station = sim.resource(2)
fuel_pump = sim.store(GAS_STATION_SIZE, GAS_STATION_SIZE)
sim.process(gas_station_control, sim, fuel_pump)
sim.process(car_generator, sim, gas_station, fuel_pump)

# Execute!
sim.run(until=SIM_TIME)


Gas Station refuelling
Car 0 arriving at gas station at 87.0
Car 0 finished refueling in 18.5 seconds.
Car 1 arriving at gas station at 129.0
Car 1 finished refueling in 19.0 seconds.
Car 2 arriving at gas station at 284.0
Car 2 finished refueling in 21.0 seconds.
Car 3 arriving at gas station at 385.0
Car 3 finished refueling in 13.5 seconds.
Car 4 arriving at gas station at 459.0
Calling tank truck at 460
Car 4 finished refueling in 22.0 seconds.
Car 5 arriving at gas station at 705.0
Car 6 arriving at gas station at 750.0
Tank truck arriving at time 760
Tank truck refuelling 188.0 liters.
Car 6 finished refueling in 29.0 seconds.
Car 5 finished refueling in 76.5 seconds.
Car 7 arriving at gas station at 891.0
Car 7 finished refueling in 13.0 seconds.


The gas station has two fuel pumps sharing a fuel tank of 200 liters. The gas station is modeled as resource (with a capacity of two). And the shared fuel tank is modeled as a store (with a capacity of 200). The cars are processes which are created by the `car_generator()` process, one at a time with some random inter-arrival time. A car arriving at the gas station first tries to acquire the a fuel pump from the gas station. Once a fuel pump is avaiable, the car gets the desired amount of gas from the fuel pump using the `get()` method, and then release the fuel pump before it leaves the gas station.

The gas station has a separate fuel level monitoring process, called `gas_station_control()`. It checks the fuel level at regular intervals (every 10 seconds). When the fuel level drops below a threshold, a tank truck will be called to refuel the tank. It takes a while for the fuel truck to arrive and when it does, it will fill up the tank using the `put()` method.

#### 4.2.3  Conditional Wait on Stores

The store facility is not a trappable. That is, you can't use it directly for conditional wait. There are two reasons. One is that a store has two methods, `get()` and `put()`, both of which could block a process. We would need to explicitly specify which of the two a process should be waiting on. The other reason is that both `get()` and `put()` methods can take arguments. The `get()` method can take the number of countable objects or the amount of uncountable quantities to be retrieved. The `put()` method can take the number or amount and also the objects themselves if so desired. 

To allow conditional wait, the store provides two methods, `getter()` and `putter()`, that return the corresponding trappable for either retrieving from the store or depositing into the store, respectively. The returned trappables can be used in a conditional wait.

In the following example, we create a scenario where a process, called `checker()`, tries to retrieve jobs from several queues (5 of them in total), one at a time. The queues are modeled as stores. The process also makes sure it checks on the time every 2 seconds. To do all these, the checker process performs a conditional wait by calling the `sim.wait()` function on the `getter()` trappables from all stores and specify a timeout using 'until'. Depending on whether timeout happens or not, the process prints on a message.

In [20]:
# %load "../examples/basics/tick.py"
from random import seed, expovariate, randrange
from sys import maxsize 
import simulus

NQ = 5 # number of queues
TKTIME = 2 # interval between clock ticks

seed(12345)

def generator():
    jobid = 0 # job id increases monotonically 
    while True:
        sim.sleep(expovariate(1))
        q = randrange(0, NQ)
        print("%g: job[%d] enters queue %d" % (sim.now, jobid, q))
        queues[q].put(obj=jobid)
        jobid += 1

def checker():
    tick = TKTIME
    while True:
        t = [q.getter() for q in queues]
        qs, timedout = sim.wait(t, until=tick, method=any)
        if timedout:
            print("%g: clock's ticking" % sim.now)
            tick += TKTIME
        else:
            q = qs.index(True)
            print("%g: job[%d] leaves queue %d" % 
                  (sim.now, t[q].retval, q))
        
sim = simulus.simulator()
queues = [sim.store(capacity=maxsize) for _ in range(NQ)]
sim.process(generator)
sim.process(checker)
sim.run(10)


0.538916: job[0] enters queue 0
0.538916: job[0] leaves queue 0
2: clock's ticking
2.25469: job[1] enters queue 2
2.25469: job[1] leaves queue 2
4: clock's ticking
4.18666: job[2] enters queue 1
4.18666: job[2] leaves queue 1
4.50171: job[3] enters queue 3
4.50171: job[3] leaves queue 3
4.67807: job[4] enters queue 0
4.67807: job[4] leaves queue 0
6: clock's ticking
6.75093: job[5] enters queue 2
6.75093: job[5] leaves queue 2
7.57665: job[6] enters queue 1
7.57665: job[6] leaves queue 1
8: clock's ticking
8.5228: job[7] enters queue 1
8.5228: job[7] leaves queue 1
8.96115: job[8] enters queue 0
8.96115: job[8] leaves queue 0
9.7184: job[9] enters queue 3
9.7184: job[9] leaves queue 3
10: clock's ticking


Note that the `getter()` trappable can keeps the returned value. In this example, the `generator()` process calls `put()` with the job's id into the store. The object can be retrieved by a consumer process when it calls `get()`, or in this case, the conditional wait puts the object in `retval` when the `getter()` trappable is triggered.

### 4.3 Mailbox

A mailbox is a facility designed specifically for message passing between processes or functions. A mailbox consists of one or more compartments or partitions. A sender can send a message to one of the partitions of the mailbox (which, by default, is partition 0) with a time delay. The message will be delivered to the designated mailbox partition at the expected time. Messages arriving at a mailbox will be stored in the individual partitions until a receiver retrieves them and removes them from the mailbox.

In Python, the concept of messages (or mails in the sense of a mailbox) takes a broader meaning.  Basically, they could be any Python objects. And since Python is a dynamically-typed language, one can also use objects of different types as messages.

A mailbox is designed to be used for both process scheduling (in process-oriented simulation) and direct-event scheduling (for event-driven simulation). In the former, a process can send as many messages to a mailbox as it needs to (by repeatedly calling the mailbox's `send()` method). The simulation time does not advance since `send()` does not block. As a matter of fact, one does not need to call `send()` in a process context at all. In this sense, a mailbox can be considered as a store with an infinite storage capacity.  An important difference, however, is that one can send messages to mailboxes and specify a different delay each time.

One or more processes can call the mailbox's `recv()` method trying to receive the messages arrived at a mailbox partition. If there are one or more messages already stored at the mailbox partition, the process will retrieve the messages from the partition and return them (in a list) without delay. Here, "retrieve" means the messages indeed will be removed from the mailbox partition.  In doing so, subsequent calls to `recv()` will no longer find the retrieved messages in the mailbox partition. If there are no messages currently in the mailbox partition when calling `recv()`, the processes will be suspended pending on the arrival of a new message to the designated mailbox partition. Once a message arrives, *all* waiting processes at the partition will be unblocked and return with the retrieved message.

There are two ways for the `recv()` method to retrieve the messages. One is to retrieve all messages from the mailbox partition; the other is to retrieve only the first one stored in the mailbox partition. The user can specify which behavior is desirable by setting the 'isall' parameter when calling the `recv()` method.  Regardless of the process trying to receive all messages or just one message, if there are multiple processes waiting to receive at a mailbox partition, it is possible a process wakes up upon a message's arrival and return empty handed (because another process may have taken the message from the mailbox). Simulus do not dictate, when multiple processes are waiting to receive a message, which one will wake up first to retrieve the messages. It'd be arbitrary since simulus handles simultaneous events without model specified ordering.

A mailbox can also be used in the direct-event scheduling context (event-driven approach). In this case, the user can add one or more callback functions to a mailbox partition (using the `add_callback()` method). A callback function can be any user-defined function. Whenever a message is delivered to a mailbox partition, *all* callback functions attached to the mailbox partition will be invoked.

Within a callback function, the user has the option to either peek at the mailbox or retrieve (or do nothing of the kind, of course).  One can call the `peek()` method to just look at the messages arrived at a mailbox partition. In this case, a list is returned containing all stored messages in the partition. The messages are not removed from the mailbox. One can also also call the `retrieve()` method. In this case, the user is given the option to retrieve just one of the messages or all messages. Again, "retrieve" means to remove the message or messages from the mailbox. Like `recv()`, the user passes in a boolean 'isall' argument to indicate which behavior is desirable.

#### 4.3.1 The PHOLD Example

The following example shows a simple use of mailbox. We use the PHOLD model, which has been commonly used for benchmarking parallel and distributed simulators. Here we create four processes, called `generate()`, each representing a node. We also create the same number of mailboxes, one for each process. The `generate()` process tries to receive a job (a message from the mailbox), one at a time, by calling the `recv()` method with 'isall' set to be False. The process then forwards the job to a randomly selected node by calling `send()` with a random delay. To start the simulation, we initially send 10 jobs to the randomly selected nodes. 

In [21]:
# %load '../examples/basics/phold.py'
import simulus

from random import seed, expovariate, randrange
seed(12345)

job_count = 5
node_count = 4
lookahead = 1

def generate(idx):
    while True:
        msg = mb[idx].recv(isall=False)
        print("%g: node %d received job[%d]," % (sim.now, idx, msg), end=' ')
        target = randrange(node_count)
        delay = expovariate(1)+lookahead
        mb[target].send(msg, delay)
        print("sent to node %d with delay %g" % (target, delay))

sim = simulus.simulator()

mb = [sim.mailbox() for _ in range(node_count)]
for idx in range(node_count):
    sim.process(generate, idx)

# disperse the initial jobs
for idx in range(job_count):
    target = randrange(node_count)
    delay = expovariate(1)+lookahead
    mb[target].send(idx, delay)
    print("init sent job[%d] to node %d with delay %g" %
          (idx, target, delay))

sim.run(5)


init sent job[0] to node 3 with delay 2.31933
init sent job[1] to node 2 with delay 2.93198
init sent job[2] to node 1 with delay 1.31505
init sent job[3] to node 3 with delay 1.17636
init sent job[4] to node 0 with delay 3.07286
1.17636: node 3 received job[3], sent to node 2 with delay 1.82572
1.31505: node 1 received job[2], sent to node 1 with delay 1.94616
2.31933: node 3 received job[0], sent to node 1 with delay 1.43835
2.93198: node 2 received job[1], sent to node 0 with delay 1.75724
3.00208: node 2 received job[3], sent to node 3 with delay 1.8796
3.07286: node 0 received job[4], sent to node 1 with delay 1.16034
3.2612: node 1 received job[2], sent to node 0 with delay 1.21069
3.75768: node 1 received job[0], sent to node 2 with delay 1.38867
4.2332: node 1 received job[4], sent to node 3 with delay 1.41479
4.47189: node 0 received job[2], sent to node 3 with delay 1.00346
4.68922: node 0 received job[1], sent to node 1 with delay 2.47331
4.88168: node 3 received job[3], sen

#### 4.3.2  Peek and Retrieve

As mentioned earlier, the `recv()` method can either retrieve all messages from the mailbox partition, or retrieve only the first one stored in the mailbox partition. In the callback function, the user can also just peek at the mailbox partion (without removing the messages), or retrieve one or all messages.

In the following example, we show a use case of these options. 

In [22]:
# %load '../examples/basics/delivery.py'
import simulus

from random import seed, expovariate, randint
seed(12345)

def generate():
    num = 0
    while True:
        sim.sleep(expovariate(1))
        print("%g: sent:" % sim.now, end=' ')
        for _ in range(randint(1,5)): # send a bunch
            print("%d" % num, end=' ')
            mb.send(num)
            num += 1
        print('')

def peek():
    msgs = mb.peek() # just peek at the mailbox
    if len(msgs) > 0:
        print("%g: peek() found:" % sim.now, end=' ')
        for m in msgs:
            print("%d" % m, end=' ')
        print('')
    else:
        print("%g: peek() found nothing" % sim.now)

def get_one():
    while True:
        sim.sleep(1)
        msg = mb.recv(isall=False)
        if msg is not None:
            print("%g: get_one() retrieved: %d" % (sim.now, msg))
        else:
            print("%g: get_one() retrieved nothing" % sim.now)

def get_all():
    while True:
        sim.sleep(5)
        msgs = mb.recv()
        if len(msgs) > 0:
            print("%g: get_all() retrieved:" % sim.now, end=' ')
            for m in msgs:
                print("%d" % m, end = ' ')
            print('')
        else:
            print("%g: get_all() retrieved nothing" % sim.now)

sim = simulus.simulator()
mb = sim.mailbox()
mb.add_callback(peek)
sim.process(generate)
sim.process(get_one)
sim.process(get_all)
sim.run(8)


0.538916: sent: 0 
0.538916: peek() found: 0 
1: get_one() retrieved: 0
2.25469: sent: 1 2 3 
2.25469: peek() found: 1 
2.25469: get_one() retrieved: 1
2.25469: peek() found: 2 
2.25469: peek() found: 2 3 
3.25469: get_one() retrieved: 2
4.18666: sent: 4 5 
4.18666: peek() found: 3 4 
4.18666: peek() found: 3 4 5 
4.25469: get_one() retrieved: 3
4.50171: sent: 6 7 8 9 
4.50171: peek() found: 4 5 6 
4.50171: peek() found: 4 5 6 9 
4.50171: peek() found: 4 5 6 9 7 
4.50171: peek() found: 4 5 6 9 7 8 
4.67807: sent: 10 
4.67807: peek() found: 4 5 6 9 7 8 10 
5: get_all() retrieved: 4 5 6 9 7 8 10 
6.75093: sent: 11 12 13 
6.75093: peek() found: 11 
6.75093: get_one() retrieved: 11
6.75093: peek() found: 12 
6.75093: peek() found: 12 13 
7.57665: sent: 14 15 
7.57665: peek() found: 12 13 14 
7.57665: peek() found: 12 13 14 15 
7.75093: get_one() retrieved: 12


In this example, we create one mailbox (with a single partition). We attach a callback function `peek()` to the mailbox (partition zero). Every time the mailbox receives a message, the `peek()` function will be invoked and the function literally just takes a peek at the mailbox and lists all the messages therein.

We create three processes. The `generate()` process sleeps for some random time and then sends a bunch of messages (ranging between 1 and 5) to the mailbox. And then it sleeps again and repeats the send. 

The `get_one()` process sleeps for 1 second (from the previous receive) and tries to retrieve one message from the mailbox. It may be blocked if the mailbox is empty. As we can see from the example, the number of messages in the mailbox seems to increase over time, since the `generate()` may send more messages than those consumed by the `get_one()` process. 

To avoid indefinite increase of the buffered messages, the `get_all()` process sleeps for 5 seconds and then tries to retrieve all messages from the mailbox in one call to `recv()` method. From the print-out, we can examine whether the behavior of the peeks and retrieves is as expected.

#### 4.3.3 Conditional Waits on Mailboxes

Similar to store, the mailbox facility is not a trappable. That is, one cannot use the mailbox directly for conditional wait. We have the same reasons. a mailbox can have multiple partitions, the `recv()` method can be blocked on receiving messages from any of the partitions. The other reason is that the `recv()` method is expected to retrieve and return one or all of the messages depending on the 'isall' argument. To allow conditional wait, mailblox provides the `receiver()` method, which returns the corresponding trappable for receiving messages from a designated mailbox partition.  

In the following example, we model the routine of a mailman and his patron. The overworked mailman is a process, who starts the day at 8 o'clock and sorts all mails until 2 o'clock in the afternoon. Then he starts to deliver the mails, which may take between one and five hours. We model this by sending a message to his patron's mailbox with a random delay. The patron is another process, who comes back from work at 5 o'clock every day. He does a timed wait on the mailbox, representing his checking the mailbox until 6 o'clock. If the mail arrives before 6, the patron receives it. Otherwise, the patron gives up and he'll receive it by next day.  

In [23]:
# %load '../examples/basics/mailman.py'
from time import gmtime, strftime
from random import seed, randint
import simulus

def strnow(t=None):
    if not t: t = sim.now
    return strftime("%H:%M", gmtime(t))

def mailman():
    day = 0
    while True:
        sim.sleep(until=day*24*3600+8*3600) # 8 o'clock
        print('--- day %d ---' % day)
        
        # sort the mails in the moring and get out for delivery at 2
        # o'clock in the afternoon
        sim.sleep(until=day*24*3600+14*3600)

        # it may take variable amount of time (between 1 to 5 hours)
        # before the mails can be delivered to people's mailboxes
        delay = randint(3600, 5*3600)
        mb.send('letter for day %d' % day, delay)
        print("%s mail truck's out, expected delivery at %s" %
              (strnow(), strnow(sim.now+delay)))

        # go to the next day
        day += 1

def patron():
    day = 0
    while True:
        # come back from work at 5 PM
        sim.sleep(until=day*24*3600+17*3600)
        
        # check the mailbox within an hour (until 6 PM)
        rcv = mb.receiver()
        _, timedout = sim.wait(rcv, 3600)
        if timedout:
            print("%s mail truck didn't come today" % strnow())
        else:
            for ltr in rcv.retval:
                print("%s receives '%s'" % (strnow(), ltr))

        # go to the next day
        day += 1
        
seed(12345)

sim = simulus.simulator()
mb = sim.mailbox()

sim.process(mailman)
sim.process(patron)

sim.run(5*24*3600)


--- day 0 ---
14:00 mail truck's out, expected delivery at 16:53
17:00 receives 'letter for day 0'
--- day 1 ---
14:00 mail truck's out, expected delivery at 18:20
18:00 mail truck didn't come today
--- day 2 ---
14:00 mail truck's out, expected delivery at 15:02
17:00 receives 'letter for day 1'
17:00 receives 'letter for day 2'
--- day 3 ---
14:00 mail truck's out, expected delivery at 18:43
18:00 mail truck didn't come today
--- day 4 ---
14:00 mail truck's out, expected delivery at 18:45
17:00 receives 'letter for day 3'
